#### Define functions and set date

In [1]:
import time
import datetime
import pandas as pd 
import pyodbc
import sys
import sqlalchemy
import nbimporter
import os
pd.set_option('display.max_columns', 25)
driver = 'ODBC Driver 17 for SQL Server'


def execute_sql(sql):

    cursor.execute(sql)
    try:
        column_list = [column[0] for column in cursor.description]
        row_list = [row for row in cursor]
        table = pd.DataFrame.from_records(row_list, columns=column_list)
        print ("Query executed.")
        return table
    except:
        print ("Query executed.")
        return
    
def push_df_sql(server, db, table_name, dataframe,option='replace'):

    """ Sends pandas dataframe to specified table in SQL Server
        sqlalchemy required b/c pyodbc in execute_sql() does not support "to sql" cmds """

    # connect to SQL Server via sqlalchemy by specifying driver/server/database
    conn_string = 'mssql+pyodbc://' + server + '/' + db + \
                    '?trusted_connection=yes&driver=' + driver

    # initialize engine to establish connection to server
    engine = sqlalchemy.create_engine(conn_string, isolation_level='READ_UNCOMMITTED')
    connection = engine.connect()

    print('Pushing df into {}.dbo.{}'.format(db, table_name))

    # push pandas df to table in sql server
    dataframe.to_sql(table_name, con=engine, if_exists=option, 
        index=False, schema=schema, chunksize=10)

    # close connection
    connection.close()

    print('--------------------')
    print('Pushed successfully')
    print('\n')
    
def push_junk(server,df,table_name,option='replace'):
    database = 'csn_junk' ####Enter database here
    
    print('Connecting to '+server+'...')
    print('---------------------')
    con_string = 'mssql+pyodbc://{0}/{1}?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server&MultiSubnetFailover=yes'.format(server,database)     
    engine = sqlalchemy.create_engine(con_string, echo=False,isolation_level='READ_UNCOMMITTED')
    con = engine.connect()
    
    print('Pushing junk table '+table_name+'...')
    print('---------------------')
    df.to_sql(name=table_name, con=engine, schema = 'dbo', if_exists=option,index=False)
    
    print('Table pushed!')
    con.close()
    engine.dispose()

def connect_db(server,db):
    import sqlalchemy
    print('Connecting to '+server+'...')
    conn = pyodbc.connect("DRIVER={" + driver + "};SERVER=" + server + "; DATABASE=" + db + "; Trusted_Connection=yes;MultiSubnetFailover=Yes",autocommit=True)
    cursor = conn.cursor()
    print ('Connected to '+server+'...')
    return conn,cursor

# Read SQL scripts in WPP folder
def read_sql(query_name):
    with open('G:\\Shared drives\\Competitive Intelligence\\8. Exclusivity\\3. Ongoing Projects\\WPP Products\\SQL\\Detections report queries\\{}'.format(query_name)) as query_file:
        return query_file.read()
    
import datetime
d = datetime.date.today()
day = d.strftime("%d")
month = d.strftime("%m")
year = d.strftime("%y")

#### Verifying month & year for tables and files

In [2]:
print("Current month set to: ",month)
a =  input('''Verify the month of the report by entering the number\n
NOTE: All existing reports/tables with same month + year will be overwritten\n''')
print(a)
if len(a) == 1:
    a = '0' + a
if a != month:
    month = a

Current month set to:  01
01


In [3]:
print("Current year set to: ",year)
a =  input('''Verify the year of the report by entering the number in YY format\n
NOTE: All existing reports/tables with same month + year will be overwritten\n''')
print(a)
if len(a) == 1:
    a = '0' + a
if len(a) == 4:
    a = a[2:]
if a != year:
    year = a

Current year set to:  20
20


#### Ensure the names of the tables are correct

In [4]:
print("Current month set to: ",month)
print("Current year set to: ",year)
#Define tables for reports
ed_table_name = 'ED_sample_{0}{1}'.format(month,year)
all_exclusive_name = 'all_exclusive_products_{0}{1}'.format(month,year)
exclusive_detections_name = 'exclusive_detection_log_{0}{1}'.format(month,year)
print("ED table name:",ed_table_name)
print("All exclusive products table name:",all_exclusive_name)
print("Violations and Detections log table name:",exclusive_detections_name)

Current month set to:  01
Current year set to:  20
ED table name: ED_sample_0120
All exclusive products table name: all_exclusive_products_0120
Violations and Detections log table name: exclusive_detection_log_0120


### Sample step

#### Pull all active agreements from EB database

In [9]:
#Get all EBs with active exclusivity into Dataframe
conn,cursor = connect_db('SQLMERCHANDISING','csn_product')
query1 = read_sql('Exclusivity Detections sampling - part 1.sql')
all_exclusive_products_df = execute_sql(query1)
conn.close()
#Gut check of all exclusive products' distribution
all_exclusive_products_df.groupby('AgreementType').nunique()

Connecting to SQLMERCHANDISING...
Connected to SQLMERCHANDISING...
Query executed.


,PrSKU,SupplierPartID,SupplierID,SupplierPartNumber,OptionCombinationID,ManufacturerPartID,SoID,ExclusivityEndDate,SupplierExclusivityPartType,AgreementType
AgreementType,,,,,,,,,,
FlagshipBrands,8236,33832,510,33736,33748,33732,1,240,5,1
LifestyleBrands,163,388,25,388,389,388,4,1,1,1
WPP,131413,379836,518,378704,379556,379089,1,408,3,1


In [11]:
#Move Dataframe to new SQLMERCHANDISING junk table
push_junk('SQLMERCHANDISING',all_exclusive_products_df,all_exclusive_name)

Connecting to SQLMERCHANDISING...
---------------------
Pushing junk table all_exclusive_products_0120...
---------------------
Table pushed!


#### Pull all Flagship Brands and Lifestyle Brands PartNumbers

In [12]:
#Connect to SQLMERCHANDISING
conn,cursor = connect_db('SQLMERCHANDISING','csn_junk')

Connecting to SQLMERCHANDISING...
Connected to SQLMERCHANDISING...


In [14]:
#SQLMERCHANDISING - get flagship brands and lifestyle brands into junk table
query2 = read_sql('Exclusivity Detections sampling - part 2 (FSB).sql').replace('mm',month).replace('yy',year) 
ed_sample_fsb = execute_sql(query2)
conn.close()
#Gut check of Flagship Brands sample distribution
ed_sample_fsb.groupby('AgreementType').nunique()

Query executed.


,PrSKU,SupplierPartID,SupplierID,SupplierPartNumber,ManufacturerPartID,OptionCombinationID,SoID,ExclusivityEndDate,SupplierExclusivityPartType,AgreementType,MatchSearch
AgreementType,,,,,,,,,,,
FlagshipBrands,8236,33832,510,33736,33732,33748,1,240,5,1,1
LifestyleBrands,163,388,25,388,388,389,4,1,1,1,1


In [15]:
#make the junk table
push_junk('SQLMERCHANDISING',ed_sample_fsb,ed_table_name)

Connecting to SQLMERCHANDISING...
---------------------
Pushing junk table ED_sample_0120...
---------------------
Table pushed!


#### Pull sample of WPP PartNumbers

In [16]:
#Connect to SQLMERCHANDISING
conn,cursor = connect_db('SQLMERCHANDISING','csn_junk')

Connecting to SQLMERCHANDISING...
Connected to SQLMERCHANDISING...


In [18]:
#WPPs
query4 = read_sql('Exclusivity Detections sampling - part 3 (WPP).sql').replace('mm',month).replace('yy',year) #SQLMERCHANDISING - get all WPP products
#insert into the junk table
ed_sample_wpp = execute_sql(query4)
#Get gut check of WPP products in sample
ed_sample_wpp.groupby('AgreementType').nunique()

Query executed.


,PrSKU,SupplierPartID,SupplierID,SupplierPartNumber,OptionCombinationID,SoID,ManufacturerPartID,ExclusivityEndDate,SupplierExclusivityPartType,AgreementType,MatchSearch
AgreementType,,,,,,,,,,,
WPP,31517,61149,429,60913,61111,1,60994,391,1,1,1


In [19]:
#Insert WPPs into sample
push_junk('SQLMERCHANDISING',ed_sample_wpp,ed_table_name,option='append')

Connecting to SQLMERCHANDISING...
---------------------
Pushing junk table ED_sample_0120...
---------------------
Table pushed!


#### Check all products in sample

In [20]:
conn,cursor = connect_db('SQLMERCHANDISING','csn_junk')
query_ED = 'select * from csn_junk..{0}'.format(ed_table_name)
table_ED = execute_sql(query_ED)
conn.close()
#Get gut check of all products in sample
table_ED.groupby('AgreementType').nunique()

Connecting to SQLMERCHANDISING...
Connected to SQLMERCHANDISING...
Query executed.


,PrSKU,SupplierPartID,SupplierID,SupplierPartNumber,ManufacturerPartID,OptionCombinationID,SoID,ExclusivityEndDate,SupplierExclusivityPartType,AgreementType,MatchSearch
AgreementType,,,,,,,,,,,
FlagshipBrands,8236,33832,510,33736,33732,33748,1,240,5,1,1
LifestyleBrands,163,388,25,388,388,389,4,1,1,1,1
WPP,31517,61149,429,60913,60994,61111,1,391,1,1,1


In [21]:
conn,cursor = connect_db('SQLMERCHANDISING','csn_junk')
query_ALL = 'select * from csn_junk..{0}'.format(all_exclusive_name)
table_ALL = execute_sql(query_ALL)
conn.close()
table_ALL.groupby('AgreementType').nunique()

Connecting to SQLMERCHANDISING...
Connected to SQLMERCHANDISING...
Query executed.


,PrSKU,SupplierPartID,SupplierID,SupplierPartNumber,OptionCombinationID,ManufacturerPartID,SoID,ExclusivityEndDate,SupplierExclusivityPartType,AgreementType
AgreementType,,,,,,,,,,
FlagshipBrands,8236,33832,510,33736,33748,33732,1,240,5,1
LifestyleBrands,163,388,25,388,389,388,4,1,1,1
WPP,131413,379836,518,378704,379556,379089,1,408,3,1


In [22]:
#Push table to SQLCOMPINTELRO
push_junk('SQLCOMPINTELRO',table_ED,ed_table_name)

Connecting to SQLCOMPINTELRO...
---------------------
Pushing junk table ED_sample_0120...
---------------------
Table pushed!


#### Backup junk tables in csv files

In [18]:
print("Does a backup ED sample table already exist for this month?")
os.path.exists("G:\\Shared drives\\Competitive Intelligence\\8. Exclusivity\\3. Ongoing Projects\\WPP Products\\ED sample backup\\{0} backup.csv".format(ed_table_name))

Does a backup ED sample table already exist for this month?


True

In [19]:
print("Does a backup table of all exclusive products already exist for this month?")
os.path.exists("G:\\Shared drives\\Competitive Intelligence\\8. Exclusivity\\3. Ongoing Projects\\WPP Products\\ED sample backup\\{0} backup.csv".format(all_exclusive_name))

Does a backup table of all exclusive products already exist for this month?


True

In [20]:
#Write backup table
table_ED.to_csv("G:\\Shared drives\\Competitive Intelligence\\8. Exclusivity\\3. Ongoing Projects\\WPP Products\\ED sample backup\\{0} backup.csv".format(ed_table_name),index=False)
table_ALL.to_csv("G:\\Shared drives\\Competitive Intelligence\\8. Exclusivity\\3. Ongoing Projects\\WPP Products\\ED sample backup\\{0} backup.csv".format(all_exclusive_name),index=False)

In [ ]:
#Read backup table
table_ED = pd.read_csv("G:\\Shared drives\\Competitive Intelligence\\8. Exclusivity\\3. Ongoing Projects\\WPP Products\\ED sample backup\\{0} backup.csv".format(ed_table_name))

#### Check how many SupplierPartIDs have detections vs not

In [5]:
conn,cursor = connect_db('SQLCOMPINTELRO','csn_junk')

Connecting to SQLCOMPINTELRO...
Connected to SQLCOMPINTELRO...


In [29]:
#How many of the PartNumbers have matches pre Matching step?
query_matches1 = '''
with cte1 as (select matWayID,matCppID 
                 from csn_comppricing..tblMatches a (nolock)
                 join csn_comppricing..tblCmpCompetitorProducts b (nolock) on matCppID = CppID AND CppCompetitorId = 115
                 where matApproved = 1),
        detected as (select distinct b.wajWayID as WayID, a.SupplierPartID,a.PrSKU, a.ExclusivityEndDate,
                     1 as MatchSearch,
                     case
                         when e.matWayID is null then 'Undetected'
                         when e.matWayID is not null then 'Detected'
                     end as DetectionStatus 
                     from csn_junk.dbo.{0} a --Changes every month
                     join csn_comppricing..tblWayfairProducts_jjIDs b (nolock) on a.JjID = b.wajJJID
                     join csn_pricing..tblActiveWayID c (nolock) on b.wajWayID = c.wayID and country = 1
                     left join cte1 e (nolock) on b.wajWayID = e.matWayID)


SELECT distinct a.SupplierPartID,DetectionStatus FROM csn_junk.dbo.{0} a
JOIN detected b ON a.SupplierPartID = b.SupplierPartID'''.format(ed_table_name)
matched_parts = execute_sql(query_matches1)
conn.close()
matched_parts.groupby('DetectionStatus').nunique()

Query executed.


,SupplierPartID,DetectionStatus
DetectionStatus,,
Detected,46555,1
Undetected,42284,1


In [9]:
#How many of the PartNumbers have matches post Matching step?
query_matches1 = '''
with cte1 as (select matManufacturerPartID as ManufacturerPartID,matCppID 
                 from csn_comppricing..tblMatches a (nolock)
                 join csn_comppricing..tblCmpCompetitorProducts b (nolock) on matCppID = CppID AND CppCompetitorId = 115
                 where matApproved = 1
                 and matClgID = 1),
        detected as (select distinct a.ManufacturerPartID, a.SupplierPartID,a.PrSKU, a.ExclusivityEndDate,
                     1 as MatchSearch,
                     case
                         when b.ManufacturerPartID is null then 'Undetected'
                         when b.ManufacturerPartID is not null then 'Detected'
                     end as DetectionStatus 
                     from csn_junk.dbo.{0} a --Changes every month
                     left join cte1 b (nolock) on a.ManufacturerPartID = b.ManufacturerPartID)


SELECT distinct a.SupplierPartID,DetectionStatus FROM csn_junk.dbo.{0} a
JOIN detected b ON a.SupplierPartID = b.SupplierPartID'''.format(ed_table_name)
matched_parts = execute_sql(query_matches1)
conn.close()
matched_parts.groupby('DetectionStatus').nunique()

Query executed.


,SupplierPartID,DetectionStatus
DetectionStatus,,
Detected,51377,1
Undetected,43898,1


#### Log all detections for the month

In [10]:
#Connect to SQLCOMPINTELRO
conn, cursor = connect_db('SQLCOMPINTELRO','csn_junk')

Connecting to SQLCOMPINTELRO...
Connected to SQLCOMPINTELRO...


In [11]:
#Compile all detections in ED sample
query9 = read_sql('Get exclusivity detections.sql').replace('mm',month).replace('yy',year)
detection_log = execute_sql(query9)
conn.close()

Query executed.


In [12]:
print("Does a backup table of all detections already exist for this month?")
os.path.exists("G:\\Shared drives\\Competitive Intelligence\\8. Exclusivity\\3. Ongoing Projects\\WPP Products\\ED sample backup\\{0} backup.csv".format(exclusive_detections_name))

Does a backup table of all detections already exist for this month?


False

In [13]:
#Write backup table
detection_log.to_csv("G:\\Shared drives\\Competitive Intelligence\\8. Exclusivity\\3. Ongoing Projects\\WPP Products\\ED sample backup\\{0} backup.csv".format(exclusive_detections_name),index=False)

In [16]:
detection_log.groupby('SoldByAmazon').nunique()

,PrSKU,SupplierPartID,SupplierID,SupplierPartNumber,ManufacturerPartID,OptionCombinationID,SoID,ExclusivityEndDate,SupplierExclusivityPartType,AgreementType,MatchSearch,WayID,DetectionDate,CrawledOn,AmazonURL,SoldByAmazon,CppManufacturer
SoldByAmazon,,,,,,,,,,,,,,,,,
0,14716,26249,527,26173,26184,26229,4,331,5,3,1,28507,1169,15,27531,1,1310
1,7536,18272,355,18220,18222,18310,5,304,5,3,1,21295,950,16,18334,1,567


In [17]:
#Push detection log into junk table in SQLMERCHANDISING
push_junk('SQLMERCHANDISING', detection_log, exclusive_detections_name)

Connecting to SQLMERCHANDISING...
---------------------
Pushing junk table exclusive_detection_log_0120...
---------------------
Table pushed!


In [18]:
#Push detection log into junk table in SQLCOMPINTELRO
push_junk('SQLCOMPINTELRO', detection_log, exclusive_detections_name)

Connecting to SQLCOMPINTELRO...
---------------------
Pushing junk table exclusive_detection_log_0120...
---------------------
Table pushed!


In [20]:
#Connect to SQLMERCHANDISING
conn, cursor = connect_db('SQLMERCHANDISING','csn_junk')

Connecting to SQLMERCHANDISING...
Connected to SQLMERCHANDISING...


### Run all reports

#### ED stats

In [21]:
query_ED_stats = read_sql('ED stats v2.sql').replace('mm',month).replace('yy',year)
ED_stats = execute_sql(query_ED_stats)

Query executed.


#### SRM report

In [22]:
query_SRM_report = read_sql('SRM report v2.sql').replace('mm',month).replace('yy',year)
SRM_report = execute_sql(query_SRM_report)

Query executed.


In [23]:
print("Does a SRM report already exist for this month?")
os.path.exists("G:\\Shared drives\\Competitive Intelligence\\8. Exclusivity\\3. Ongoing Projects\\WPP Products\\Reporting\\{0}-{1} Report\\SRM Report {0}-{1}.xlsx".format(month,year))

Does a SRM report already exist for this month?


False

In [25]:
SRM_report.to_excel("G:\\Shared drives\\Competitive Intelligence\\8. Exclusivity\\3. Ongoing Projects\\WPP Products\\Reporting\\{0}-{1} Report\\SRM Report {0}-{1}.xlsx".format(month,year),index=False)

#### Flagship Brands report

In [27]:
query_FB_report = read_sql('Flagship Brands report v2.sql').replace('mm',month).replace('yy',year)
FB_report = execute_sql(query_FB_report)

Query executed.


#### Marketing Category Breakdown

In [28]:
query_MC_report = read_sql('Marketing Categories report v2.sql').replace('mm',month).replace('yy',year)
MC_report = execute_sql(query_MC_report)

Query executed.


In [31]:
query_avg_report = read_sql('Average GRS report v2.sql').replace('mm',month).replace('yy',year)
avg_report = execute_sql(query_avg_report)
conn.close()

Query executed.


#### Write reports to a file

In [32]:
print("Does a report already exist for this month?")
os.path.exists("G:\\Shared drives\\Competitive Intelligence\\8. Exclusivity\\3. Ongoing Projects\\WPP Products\\Reporting\\{0}-{1} Report\\Exclusivity Detections Report {0}-{1}.xlsx".format(month,year))

Does a report already exist for this month?


False

In [33]:
import os
os.chdir("G:\\Shared drives\\Competitive Intelligence\\8. Exclusivity\\3. Ongoing Projects\\WPP Products\\Reporting\\{0}-{1} Report".format(month,year))
with pd.ExcelWriter("Exclusivity Detections Report {0}-{1}.xlsx".format(month,year)) as writer:
    ED_stats.to_excel(writer, sheet_name='Overall',index=False)
    SRM_report.to_excel(writer, sheet_name='List of Violations',index=False)
    FB_report.to_excel(writer,sheet_name='Flagship Brands',index=False)
    MC_report.to_excel(writer,sheet_name='Marketing Categories',index=False)
    avg_report.to_excel(writer,sheet_name='Average GRS',index=False)